In [39]:
import cuspatial
import cudf
import cupy
import geopandas
import time
import pandas as pd
import numpy as np
from shapely.geometry import *
from shapely import wkt

# GeoPandas - Load data

Load from pandas df: https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html

Load data from file: https://geopandas.org/en/stable/getting_started/introduction.html

## point

In [40]:
df = pd.DataFrame(
    {
        "City": ["Buenos Aires", "Brasilia", "Santiago", "Bogota", "Caracas"],
        "Country": ["Argentina", "Brazil", "Chile", "Colombia", "Venezuela"],
        "Latitude": [-34.58, -15.78, -33.45, 4.60, 10.48],
        "Longitude": [-58.66, -47.91, -70.66, -74.08, -66.86],
    }
)

In [41]:
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude), crs="EPSG:4326"
)

In [42]:
gdf

,City,Country,Latitude,Longitude,geometry
0,Buenos Aires,Argentina,-34.58,-58.66,POINT (-58.66000 -34.58000)
1,Brasilia,Brazil,-15.78,-47.91,POINT (-47.91000 -15.78000)
2,Santiago,Chile,-33.45,-70.66,POINT (-70.66000 -33.45000)
3,Bogota,Colombia,4.60,-74.08,POINT (-74.08000 4.60000)
4,Caracas,Venezuela,10.48,-66.86,POINT (-66.86000 10.48000)


### using wkt

In [43]:
point_df = pd.DataFrame(
    {
        "City": ["Buenos Aires", "Brasilia", "Santiago", "Bogota", "Caracas"],
        "Country": ["Argentina", "Brazil", "Chile", "Colombia", "Venezuela"],
        "geometry": [
            "POINT(-58.66 -34.58)",
            "POINT(-47.91 -15.78)",
            "POINT(-70.66 -33.45)",
            "POINT(-74.08 4.60)",
            "POINT(-66.86 10.48)",
        ],
    }
)

In [44]:
point_df["geometry"] = geopandas.GeoSeries.from_wkt(point_df["geometry"])
point_gdf = geopandas.GeoDataFrame(point_df, geometry="geometry")

print(point_gdf.head())

           City    Country                     geometry
0  Buenos Aires  Argentina  POINT (-58.66000 -34.58000)
1      Brasilia     Brazil  POINT (-47.91000 -15.78000)
2      Santiago      Chile  POINT (-70.66000 -33.45000)
3        Bogota   Colombia    POINT (-74.08000 4.60000)
4       Caracas  Venezuela   POINT (-66.86000 10.48000)


In [45]:
print(type(point_gdf['geometry'][0]))
print(point_gdf['geometry'][0].xy)
print(point_gdf['geometry'][0].xy[0][0])
print(point_gdf['geometry'][0].xy[1][0])

<class 'shapely.geometry.point.Point'>
(array('d', [-58.66]), array('d', [-34.58]))
-58.66
-34.58


In [46]:
def get_point_xy(point):
    """
    Input: 
        type: shapely.geometry.polygon.Point, e.g., point_gdf['geometry'][0]
    Output:
        x, y
    """
    x = point.xy[0][0]
    y = point.xy[1][0]
    return x, y

In [47]:
def get_point_boundaries(point_column):
    """
    Input: 
        a list (column) of shapely.geometry.polygon.Point, e.g., point_gdf['geometry']
    Output:
        x_min, x_max, y_min, y_max
    """
    x_list_all, y_list_all = [], []
    for point in point_column:
        x, y = get_point_xy(point)
        x_list_all.append(x)
        y_list_all.append(y)
    x_min = np.amin(x_list_all)
    x_max = np.amax(x_list_all)
    y_min = np.amin(y_list_all)
    y_max = np.amax(y_list_all)
    return x_min, x_max, y_min, y_max

In [48]:
get_point_xy(point_gdf['geometry'][0])

(-58.66, -34.58)

In [49]:
get_point_boundaries(point_gdf['geometry'])

(-74.08, -47.91, -34.58, 10.48)

## Polygon with WKT

In [50]:
polygon_df = pd.DataFrame(
    {
        "City": ["Buenos Aires", "Brasilia", "Santiago", "Bogota", "Caracas"],
        "Country": ["Argentina", "Brazil", "Chile", "Colombia", "Venezuela"],
        "geometry": [
            "POLYGON((13.7398334 51.0523423, 13.7398334 51.0532229, 13.741235 51.0532229, 13.741235 51.0523423, 13.7398334 51.0523423))",
            "POLYGON((13.7371957 51.0487779, 13.7371957 51.0489591, 13.7375362 51.0489591, 13.7375362 51.0487779, 13.7371957 51.0487779))",
            "POLYGON((24.9524119 60.1720899, 24.9524119 60.1722463, 24.9527577 60.1722463, 24.9527577 60.1720899, 24.9524119 60.1720899))",
            "POLYGON((8.9897244 48.6797103, 8.9897244 48.6810064, 8.9920898 48.6810064, 8.9920898 48.6797103, 8.9897244 48.6797103))",
            "POLYGON((27.4414033 53.9137695, 27.4414033 53.9139195, 27.4415883 53.9139195, 27.4415883 53.9137695, 27.4414033 53.9137695))",
        ],
    }
)

In [51]:
polygon_df["geometry"] = geopandas.GeoSeries.from_wkt(polygon_df["geometry"])
polygon_gdf = geopandas.GeoDataFrame(polygon_df, geometry="geometry")
print(polygon_gdf.head())

           City    Country                                           geometry
0  Buenos Aires  Argentina  POLYGON ((13.73983 51.05234, 13.73983 51.05322...
1      Brasilia     Brazil  POLYGON ((13.73720 51.04878, 13.73720 51.04896...
2      Santiago      Chile  POLYGON ((24.95241 60.17209, 24.95241 60.17225...
3        Bogota   Colombia  POLYGON ((8.98972 48.67971, 8.98972 48.68101, ...
4       Caracas  Venezuela  POLYGON ((27.44140 53.91377, 27.44140 53.91392...


In [52]:
print(type(polygon_gdf['geometry'][0]))
print(polygon_gdf['geometry'][0])
print(polygon_gdf['geometry'][0].exterior.coords.xy)

<class 'shapely.geometry.polygon.Polygon'>
POLYGON ((13.7398334 51.0523423, 13.7398334 51.0532229, 13.741235 51.0532229, 13.741235 51.0523423, 13.7398334 51.0523423))
(array('d', [13.7398334, 13.7398334, 13.741235, 13.741235, 13.7398334]), array('d', [51.0523423, 51.0532229, 51.0532229, 51.0523423, 51.0523423]))


In [53]:
for e in polygon_gdf['geometry'][0].exterior.coords.xy[0]:
    print(e)

13.7398334
13.7398334
13.741235
13.741235
13.7398334


In [54]:
for e in polygon_gdf['geometry'][0].exterior.coords.xy[1]:
    print(e)

51.0523423
51.0532229
51.0532229
51.0523423
51.0523423


In [55]:
def get_polygon_xy_list(polygon):
    """
    Input: 
        type: shapely.geometry.polygon.Polygon, e.g., polygon_gdf['geometry'][0]
    Output:
        x_list, y_list. Each is a list of coordinates on x and y.
    """
    x_list = []
    y_list = []
    for e in polygon.exterior.coords.xy[0]:
        x_list.append(e)
    for e in polygon.exterior.coords.xy[1]:
        y_list.append(e)
    return x_list, y_list

In [56]:
def get_polygon_boundaries(polygon_column):
    """
    Input: 
        a list (column) of shapely.geometry.polygon.Polygon, e.g., polygon_gdf['geometry']
    Output:
        x_min, x_max, y_min, y_max
    """
    x_list_all, y_list_all = [], []
    for polygon in polygon_column:
        x_list, y_list = get_polygon_xy_list(polygon)
        x_list_all += x_list
        y_list_all += y_list
    x_min = np.amin(x_list_all)
    x_max = np.amax(x_list_all)
    y_min = np.amin(y_list_all)
    y_max = np.amax(y_list_all)
    return x_min, x_max, y_min, y_max

In [57]:
get_polygon_xy_list(polygon_gdf['geometry'][0])

([13.7398334, 13.7398334, 13.741235, 13.741235, 13.7398334],
 [51.0523423, 51.0532229, 51.0532229, 51.0523423, 51.0523423])

In [58]:
x_min, x_max, y_min, y_max = get_polygon_boundaries(polygon_gdf['geometry'])
print("x_min, x_max, y_min, y_max: ", x_min, x_max, y_min, y_max)

x_min, x_max, y_min, y_max:  8.9897244 27.4415883 48.6797103 60.1722463


## Load from CSV

In [59]:
def load_geodf(path):
    """
    Load a csv file first as pandas dataframe, then convert it to geopandas df.
    Note: the input file must has the geometry column
    """
    dtype = {"geometry": str}
    df = pd.read_csv(path, dtype=dtype)
    df["geometry"] = geopandas.GeoSeries.from_wkt(df["geometry"])
    gdf = geopandas.GeoDataFrame(df, geometry="geometry")
    
    return gdf

In [60]:
point_gdf = load_geodf("./files/points.csv")
print(point_gdf.head())

   gid                   geometry
0    1  POINT (6964.700 1060.660)
1    2  POINT (2861.400 7454.720)
2    3  POINT (2268.520 5723.150)
3    4  POINT (5513.160 4582.420)
4    5  POINT (7194.700 3847.070)


In [61]:
polygon_gdf = load_geodf("./files/polygons.csv")
print(polygon_gdf.head())

   gid                                           geometry
0    1  POLYGON ((6964.690 1060.650, 6964.690 1061.650...
1    2  POLYGON ((2861.390 7454.710, 2861.390 7455.710...
2    3  POLYGON ((2268.510 5723.140, 2268.510 5724.140...
3    4  POLYGON ((5513.150 4582.410, 5513.150 4583.410...
4    5  POLYGON ((7194.690 3847.060, 7194.690 3848.060...


# Join example

In [62]:
polygon_df = pd.DataFrame(
    {
        "geometry": [
			"POLYGON((6964.69 1060.65, 6964.69 1061.65, 6965.69 1061.65, 6965.69 1060.65, 6964.69 1060.65))",
			"POLYGON((2861.39 7454.71, 2861.39 7455.71, 2862.39 7455.71, 2862.39 7454.71, 2861.39 7454.71))",
			"POLYGON((2268.51 5723.14, 2268.51 5724.14, 2269.51 5724.14, 2269.51 5723.14, 2268.51 5723.14))",
			"POLYGON((5513.15 4582.41, 5513.15 4583.41, 5514.15 4583.41, 5514.15 4582.41, 5513.15 4582.41))",
			"POLYGON((7194.69 3847.06, 7194.69 3848.06, 7195.69 3848.06, 7195.69 3847.06, 7194.69 3847.06))",
        ],
    }
)

polygon_df["geometry"] = geopandas.GeoSeries.from_wkt(polygon_df["geometry"])
polygon_gdf = geopandas.GeoDataFrame(polygon_df, geometry="geometry")
print(polygon_gdf)

                                            geometry
0  POLYGON ((6964.690 1060.650, 6964.690 1061.650...
1  POLYGON ((2861.390 7454.710, 2861.390 7455.710...
2  POLYGON ((2268.510 5723.140, 2268.510 5724.140...
3  POLYGON ((5513.150 4582.410, 5513.150 4583.410...
4  POLYGON ((7194.690 3847.060, 7194.690 3848.060...


In [63]:
point_df = pd.DataFrame(
    {
        "geometry": [
			# left lower corner + 0.01
            "POINT(6964.70 1060.66)",
            "POINT(2861.40 7454.72)",
            "POINT(2268.52 5723.15)",
            "POINT(5513.16 4582.42)",
            "POINT(7194.70 3847.07)",
            "POINT(7194.71 3847.08)",

			# original left lower corner
            # "POINT(6964.69 1060.65)",
            # "POINT(2861.39 7454.71)",
            # "POINT(2268.51 5723.14)",
            # "POINT(5513.15 4582.41)",
            # "POINT(7194.69 3847.06)",
        ],
    }
)

point_df["geometry"] = geopandas.GeoSeries.from_wkt(point_df["geometry"])
point_gdf = geopandas.GeoDataFrame(point_df, geometry="geometry")
print(point_gdf)

                    geometry
0  POINT (6964.700 1060.660)
1  POINT (2861.400 7454.720)
2  POINT (2268.520 5723.150)
3  POINT (5513.160 4582.420)
4  POINT (7194.700 3847.070)
5  POINT (7194.710 3847.080)


In [64]:
gpu_polygon_df = cuspatial.from_geopandas(polygon_gdf)
gpu_point_df = cuspatial.from_geopandas(point_gdf)
polygons_gpu = gpu_polygon_df['geometry']
points_gpu = gpu_point_df['geometry']
print(type(polygons_gpu))
print(type(points_gpu))

<class 'cuspatial.core.geoseries.GeoSeries'>
<class 'cuspatial.core.geoseries.GeoSeries'>


In [65]:
points_gpu

0    POINT (6964.700 1060.660)
1    POINT (2861.400 7454.720)
2    POINT (2268.520 5723.150)
3    POINT (5513.160 4582.420)
4    POINT (7194.700 3847.070)
5    POINT (7194.710 3847.080)
Name: geometry, dtype: geometry

In [66]:
polygons_gpu

0    POLYGON ((6964.690 1060.650, 6964.690 1061.650...
1    POLYGON ((2861.390 7454.710, 2861.390 7455.710...
2    POLYGON ((2268.510 5723.140, 2268.510 5724.140...
3    POLYGON ((5513.150 4582.410, 5513.150 4583.410...
4    POLYGON ((7194.690 3847.060, 7194.690 3848.060...
Name: geometry, dtype: geometry

## Unindexed Join

In [67]:
points_in_polygon = cuspatial.point_in_polygon(
    points_gpu, polygons_gpu
)
print(points_in_polygon.head())

       0      1      2      3      4
0   True  False  False  False  False
1  False   True  False  False  False
2  False  False   True  False  False
3  False  False  False   True  False
4  False  False  False  False   True


In [68]:
sum_of_points_in_polygons = points_in_polygon.sum()
print(sum_of_points_in_polygons)
print('total pairs:', sum_of_points_in_polygons.sum())

0    1
1    1
2    1
3    1
4    2
dtype: int64
total pairs: 6


Seems point intersection does not count in cuspatial. 

## Indexed Join

Procedure:
1. Construct a quadtree based on the points
2. Intersect the polygons and the quadtree leaves, record all intersection
3. For each leaf, nested loop join between the points and the intersected polygons

Seems all steps happen on GPU thanks to the very regular quadtree architecture. 


In [69]:
"""
x_min, x_max, y_min, y_max, scale must be consistent on quadtree construction (quadtree_on_points) and join (join_quadtree_and_bounding_boxes)
There official document uses different values thus lead to bugs...
"""

point_x_min, point_x_max, point_y_min, point_y_max = get_point_boundaries(point_gdf['geometry'])
polygon_x_min, polygon_x_max, polygon_y_min, polygon_y_max = get_polygon_boundaries(polygon_gdf['geometry'])
x_min = np.amin([point_x_min, polygon_x_min])
x_max = np.amax([point_x_max, polygon_x_max])
y_min = np.amin([point_y_min, polygon_y_min])
y_max = np.amax([point_y_max, polygon_y_max])
print(point_x_min, point_x_max, point_y_min, point_y_max)
print(polygon_x_min, polygon_x_max, polygon_y_min, polygon_y_max)

2268.52 7194.71 1060.66 7454.72
2268.51 7195.69 1060.65 7455.71


quadtree_on_points

https://github.com/rapidsai/cuspatial/blob/branch-23.06/python/cuspatial/cuspatial/core/spatial/indexing.py#L15

Seems the index is constructed on GPU as well:
https://github.com/rapidsai/cuspatial/blob/branch-23.06/cpp/src/indexing/point_quadtree.cu 


The function uses a set of points and a user-defined bounding box to build an
indexing quad tree. Be sure to adjust the parameters appropriately, with larger
parameter values for larger datasets.

scale: A scaling function that increases the size of the point space from an
origin defined by {x_min, y_min}. This can increase the likelihood of generating
well-separated quads.

max_depth: In order for a quadtree to index points effectively, it must have a depth that is log-scaled with the size of the number of points. Each level of the quad tree contains 4 quads. The number of available quads for indexing is then equal to 4^d where d is the max_depth parameter. With an input size
of 10m points and max_depth = 7, 1e7/(4^7)=610 points will be most efficiently packed into the leaves of the quad tree.

max_size: The maximum number of points allowed in an internal node before it is split into four leaf notes. As the quadtree is generated, a leaf node containing usable index points will be created as points are added. If the number of points in this leaf exceeds max_size, the leaf will be subdivided, with four new leaves added and the original node removed from the set of leaves. This number is probably optimized in most datasets by making it a significant fraction of the optimal leaf size computation from above. Consider leaf_size / 4 -> 1e7/(4^7)/4=153.

Wenqi: they set the min scale as `min_scale = max(x_max - x_min, y_max - y_min) / ((1 << max_depth) + 2)`, we can just reuse this setup.

Wenqi: maybe, according to this example, a good method is to, given a dataset size, first choose the maximum level by constraining the average number of points per leaf. Then, we set the max_size of inner nodes, which can be 1/4 of the average number of points per leaf.

In [70]:
def get_tree_parameters(n_points, avg_leaf_size, x_min, x_max, y_min, y_max):
    """
    Given the number of points to index, and an average leaf_size, return the tree settings:
        scale (some normalization parameters), max_depth (tree depth), max_size (inner node size)
    """
    max_depth = 2
    while True:
        if avg_leaf_size * (4 ** max_depth) >= n_points:
            break
        else:
            max_depth += 1
    
    max_size = avg_leaf_size // 4
    scale = int(max(x_max - x_min, y_max - y_min) / ((1 << max_depth) + 2)) * 10
    print('Set the scale to the default x10, otherwise there are cases where results are 0 on large datasets')
    # scale = int(max(x_max - x_min, y_max - y_min) / ((1 << max_depth) + 2)) + 1
    
    return scale, max_depth, max_size
    

In [71]:
# quadtree on points
# Note, this code block cannot be run repetitively, because the older index will already reside in GPU memory. Rerun the ipynb kernel before rerun this block
n_points = point_gdf['geometry'].count()
avg_leaf_size = 1024
scale, max_depth, max_size = get_tree_parameters(n_points, avg_leaf_size, x_min, x_max, y_min, y_max)
# scale = 1
# max_depth = 7
# max_size = 125
print(f"scale: {scale} max_depth: {max_depth} max_size: {max_size}")
point_indices, quadtree = cuspatial.quadtree_on_points(points_gpu,
                                                       x_min,
                                                       x_max,
                                                       y_min,
                                                       y_max,
                                                       scale,
                                                       max_depth,
                                                       max_size)

Set the scale to the default x10, otherwise there are cases where results are 0 on large datasets
scale: 10650 max_depth: 2 max_size: 256


In [72]:
print(point_indices.head())
print(quadtree.head())

0    0
1    1
2    2
3    3
4    4
dtype: uint32
   key  level  is_internal_node  length  offset
0    0      0             False       6       0


join_quadtree_and_bounding_boxes

https://github.com/rapidsai/cuspatial/blob/branch-23.06/python/cuspatial/cuspatial/core/spatial/join.py#L98

Search a quadtree for polygon or linestring bounding box intersections.

result : cudf.DataFrame
	Indices for each intersecting bounding box and leaf quadrant.

	bbox_offset : cudf.Series
		Indices for each bbox that intersects with the quadtree.
	quad_offset : cudf.Series
		Indices for each leaf quadrant intersecting with a poly bbox.

CUDA kernel:

https://github.com/rapidsai/cuspatial/blob/branch-23.06/cpp/src/join/quadtree_bbox_filtering.cu

In [73]:
poly_bboxes = cuspatial.polygon_bounding_boxes(
    polygons_gpu
)
print(quadtree)
print(poly_bboxes)
print(scale)
print(max_depth)
# Intersects the polygon bounding box and the quad tree
intersections = cuspatial.join_quadtree_and_bounding_boxes(
    quadtree,
    poly_bboxes,
    x_min,
    x_max,
    y_min,
    y_max,
    scale,
    max_depth
)
print(intersections)

   key  level  is_internal_node  length  offset
0    0      0             False       6       0
      minx     miny     maxx     maxy
0  6964.69  1060.65  6965.69  1061.65
1  2861.39  7454.71  2862.39  7455.71
2  2268.51  5723.14  2269.51  5724.14
3  5513.15  4582.41  5514.15  4583.41
4  7194.69  3847.06  7195.69  3848.06
10650
2
   bbox_offset  quad_offset
0            0            0
1            1            0
2            2            0
3            3            0
4            4            0


quadtree_point_in_polygon

https://github.com/rapidsai/cuspatial/blob/branch-23.06/python/cuspatial/cuspatial/core/spatial/join.py#L165

    result : cudf.DataFrame
        Indices for each intersecting point and polygon pair.

CUDA kernel:

https://github.com/rapidsai/cuspatial/blob/branch-23.06/cpp/src/join/quadtree_point_in_polygon.cu 

In [74]:
# Only the points are indexed, not the polygons
start = time.time()
polygons_and_points = cuspatial.quadtree_point_in_polygon(
    intersections,
    quadtree,
    point_indices,
    points_gpu,
    polygons_gpu
)
end = time.time()
print('total join pairs:', polygons_and_points['polygon_index'].count())
print('time: {} ms'.format(1000 * (end - start)))
print(polygons_and_points)

total join pairs: 6
time: 34.20400619506836 ms
   polygon_index  point_index
0              0            0
1              1            1
2              2            2
3              3            3
4              4            4
5              4            5


## Batch Indexed Join

In [75]:
# batch processing of polygon dataset
batch_size = 1
batch_num = polygon_gdf['geometry'].count()
join_count = 0
for batch_id in range(batch_num):

	print('batch: ', batch_id)
	start_id = batch_id * batch_size
	end_id = (batch_id + 1) * batch_size
	# if end_id >= polygon_gdf['geometry'].count():
	#     end_id = polygon_gdf['geometry'].count() - 1

	print(polygon_gdf['geometry'][start_id: end_id])
	
	# polygon_batch = polygons_gpu
	polygon_batch = polygons_gpu[start_id: end_id]
	print(polygon_batch)
	# polygon intersect index
	# poly_bboxes = cuspatial.polygon_bounding_boxes(polygon_batch)
	poly_bboxes = cuspatial.polygon_bounding_boxes(polygon_batch)
	print(poly_bboxes)
	intersections = cuspatial.join_quadtree_and_bounding_boxes(
		quadtree,
		poly_bboxes,
		x_min,
		x_max,
		y_min,
		y_max,
		scale,
		max_depth
	)
	print(intersections)
	polygons_and_points = cuspatial.quadtree_point_in_polygon(
		intersections,
		quadtree,
		point_indices,
		points_gpu,
		polygon_batch
	)
	print(polygons_and_points['polygon_index'].count())
	join_count += polygons_and_points['polygon_index'].count()

batch:  0
0    POLYGON ((6964.690 1060.650, 6964.690 1061.650...
Name: geometry, dtype: geometry
0    POLYGON ((6964.690 1060.650, 6964.690 1061.650...
Name: geometry, dtype: geometry
      minx     miny     maxx     maxy
0  6964.69  1060.65  6965.69  1061.65
   bbox_offset  quad_offset
0            0            0
1
batch:  1
1    POLYGON ((2861.390 7454.710, 2861.390 7455.710...
Name: geometry, dtype: geometry
1    POLYGON ((2861.390 7454.710, 2861.390 7455.710...
Name: geometry, dtype: geometry
      minx     miny     maxx     maxy
0  2861.39  7454.71  2862.39  7455.71
   bbox_offset  quad_offset
0            0            0
1
batch:  2
2    POLYGON ((2268.510 5723.140, 2268.510 5724.140...
Name: geometry, dtype: geometry
2    POLYGON ((2268.510 5723.140, 2268.510 5724.140...
Name: geometry, dtype: geometry
      minx     miny     maxx     maxy
0  2268.51  5723.14  2269.51  5724.14
   bbox_offset  quad_offset
0            0            0
1
batch:  3
3    POLYGON ((5513.150 4582.410, 55

In [76]:
join_count

6